In [0]:


import psycopg2
import datetime
from applications.db_check_v1.cases.store.market_size_v1.constants import MARKET_SIZE_DSN

today = datetime.datetime.today()
date_delta = z.input("max_days")
date_start = today + datetime.timedelta(0-int(date_delta)-1)


sql = 'SELECT date,device_code,count(*) FROM store.store_market_size_fact_v1  WHERE ( date between \'{date_start}\' AND \'{today}\'  ) GROUP BY date, device_code ORDER BY date ASC'.format(date_start=date_start, today=today)

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
test_result = query(MARKET_SIZE_DSN, sql)
print "%table {}\t{}\t{}".format( "date", "device_code", "count")

for result in test_result:
    date,device_code,count = result
    print "{}\t{}\t{}".format(date.strftime("%Y-%m-%d"), device_code, count)



In [0]:
%%sh

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 

SELECT * from store.store_market_size_latest_date_v1 limit 100;

EOF

In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2020-01-10' and 
        purchase_type_id=12 and 
        app_type_id=2 and 
        store_id=14 and 
        category_id in (65) 
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF



In [0]:
%%sh
#aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-26/platform=android/
#aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-28/platform=android/
#aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-01/platform=android/

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-17/device_code=android-all/

In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/code.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)


In [0]:


"""
DB Check modules
"""

import datetime
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, etl_skip
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING, \
    CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE as CATEGORY_ID_MAPPING
from applications.db_check_v1.common.utils import get_week_start_end_date, get_date_list
from applications.db_check_v1.cases.store.market_size_v1.constants import MARKET_SIZE_DSN


class MarketSizeRawData(object):
    raw_s3_path = "s3://b2c-prod-dca-store-estimates/store_estv2/MARKET_SIZE_ESTIMATES_FINAL/" \
                  "version=2.0.0/range_type=DAY/date={date}/"
    device_code_mapping = {
        "0google-play": "android-all",
        "1ios": "ios-all",
        "1ipad": "ios-tablet",
        "1iphone": "ios-phone",
    }

    metric_mapping = {
        "downloads": "est_market_size_download",
        "revenue": "est_market_size_revenue"
    }

    dimension_mapping = {
        "price_type": "app_price_type_id",
        "purchase_type": "purchase_type_id"
    }

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        df = self._get_raw_data_by_date(date)
        df = self._parse_mapping(df)
        df = self._parse_unified_format(df)
        df = self._data_clean_up(df)
        return df

    def _data_clean_up(self, df):
        # clean unknown mapping
        category_id_list = list(set(CATEGORY_ID_MAPPING["apple-store"]["ios-all"].values() +
                                    CATEGORY_ID_MAPPING["google-play"]["android-all"].values()))

        country_code_list = list(set(COUNTRY_CODE_MAPPING["apple-store"].values() +
                                     COUNTRY_CODE_MAPPING["google-play"].values()))

        df = df[(df['category_id'].isin(category_id_list)) & (df['country_code'].isin(country_code_list))]
        return df

    def _parse_mapping(self, df):
        # country_code mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"store_id": COUNTRY_CODE_MAPPING["google-play"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"store_id": COUNTRY_CODE_MAPPING["apple-store"]})
        df = df.rename(columns={'store_id': 'country_code'})

        # category_id mapping
        df.loc[df["platform_id"] == 0] = df.loc[df["platform_id"] == 0].\
            replace({"category_id": CATEGORY_ID_MAPPING["google-play"]["android-all"]})
        df.loc[df["platform_id"] == 1] = df.loc[df["platform_id"] == 1].\
            replace({"category_id": CATEGORY_ID_MAPPING["apple-store"]["ios-all"]})

        # device_code mapping
        df["device_code"] = df["platform_id"].astype(str) + df["device"]
        df = df.replace({"device_code": self.device_code_mapping})
        return df

    def _parse_unified_format(self, df):
        df = df.rename(columns=self.dimension_mapping)
        df = df.pivot_table(index=["app_price_type_id", "purchase_type_id", "category_id",
                                   "device_code", "country_code"], columns='data_type', values='estimate')
        df.reset_index(inplace=True)
        df.columns.name = None
        df = df.rename(columns=self.metric_mapping)
        return df

    def _get_raw_data_by_date(self, date):
        """
        +--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
        |store_id|      date|platform_id|     device|data_type|price_type|purchase_type|category_id|estimate|
        +--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
        |      10|2020-01-10|          0|google-play|downloads|         1|           10|          1|11981138|
        +--------+----------+-----------+-----------+---------+----------+-------------+-----------+--------+
        """
        # schema = StructType([
        #     StructField("store_id", IntegerType(), False),
        #     StructField("date", StringType(), False),
        #     StructField("platform_id", IntegerType(), False),
        #     StructField("device", StringType(), False),
        #     StructField("data_type", StringType(), False),
        #     StructField("price_type", IntegerType(), False),
        #     StructField("purchase_type", IntegerType(), False),
        #     StructField("category_id", IntegerType(), False),
        #     StructField("estimate", LongType(), False)
        # ])
        raw_df = self.spark.read.parquet(self.raw_s3_path.format(date=date))
        return raw_df.toPandas()

    def get_rank_count_and_sum_by_date(self, date):
        pass
        # return raw_count, raw_sum


class MarketSizeUnifiedData(object):
    unified_s3_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/" \
                      "granularity=daily/date={}/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        #unified_df = self.spark.read.parquet(self.unified_s3_path.format(date)).toPandas()
        
        unified_df = self.spark.read.format("delta").load(self.unified_s3_path.format(date)).toPandas()
        unified_df = unified_df.drop(["_identifier"], axis=1)
        return unified_df


class MarketSizeDBData(object):
    def get(self, date):
        sql = "SELECT * FROM store.store_market_size_fact_v1 WHERE date='{}'".format(date)
        return query_df(MARKET_SIZE_DSN, sql)

    def get_led(self):
        sql = "SELECT * FROM store.store_market_size_latest_date_v1"
        return query_df(MARKET_SIZE_DSN, sql)


class TestMarketSizeWeekly(PipelineTest):
    # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
    trigger_date_config = ('* 16 * * 2', 3)

    def _compare_df(self, df1, df2, log=''):
        len_diff = 0
        for diff_type in ["left", "right"]:
            diff_df = df1.merge(df2, indicator=True, how=diff_type)  # .loc[lambda x : x['_merge']!='both']
            diff_df = diff_df.loc[diff_df["_merge"] != "both"]
            len_diff = len(diff_df) if len(diff_df)>len_diff else len_diff
            if len(diff_df) != 0:
                print diff_type
                print "dataframe overview of df1 and df2"
                print diff_df
                print "dimension overview of diff df"
                print diff_df.country_code.unique()
                print diff_df.category_id.unique()
                print diff_df.device_code.unique()
            print "found mismatch when compare the raw, unified, db. diff count is \n {}, logs:{}".format(len(diff_df), log)
            # self.assertEqual(len(diff_df), 0,
            #                  msg="found mismatch when compare the raw, unified, db."
            #                      " diff count is \n {}, logs:{}".format(len(diff_df), log))

    @etl_skip()
    def test_market_size_etl_accuracy_and_completeness(self):
        # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
        start_date, end_date = get_week_start_end_date(self.check_date_str)
        date_list = get_date_list(start_date, end_date)
        for date in date_list:
            print "************** {} *************".format(date)
            raw_df = MarketSizeRawData(self.spark).get(date)
            unified_df = MarketSizeUnifiedData(self.spark).get(date)
            db_df = MarketSizeDBData().get(date)

            self._compare_df(raw_df, unified_df, log="raw / unified - {}".format(date))
            self._compare_df(unified_df, db_df, log="unified / db - {}".format(date))
        raise RuntimeError(date_list)

    def test_market_size_etl_timelines(self):
        # Every Tuesday 16:00 UTC time will refresh the data of last Full Week.
        # E.g. 2020-02-11 17:00 the data of 2020-02-02 ~ 2020-02-08 will be ready
        trigger_datetime = datetime.datetime.strptime("2020-02-11 17:00:00", '%Y-%m-%d %H:%M:%S')
        check_date_str_actual = self._get_check_date_from_routing_config(trigger_datetime).strftime("%Y-%m-%d")
        self.assertEqual("2020-02-08", check_date_str_actual)

    @etl_skip()
    def test_market_size_led(self):
        _, expected_led_date = get_week_start_end_date(self.check_date_str)
        led_df = MarketSizeDBData().get_led()
        for device_code in ["ios-all", "android-all", "ios-tablet", "ios-phone"]:
            actual_led_date = led_df.loc[led_df.device_code == device_code].date.values[0].strftime("%Y-%m-%d")
            self.assertEqual(expected_led_date, actual_led_date,
                             msg="current led for device : {} is {}, it should be {}".format(
                                 device_code, actual_led_date, expected_led_date))


In [0]:


import sys
import datetime
import traceback
import unittest
from applications.db_check_v1.common.html_report_test_runner import HTMLTestRunner


def debug(case_list):
    std_out_origin= sys.stdout
    std_err_origin= sys.stderr
    try:
        suite =  unittest.TestSuite()
        for case in case_list:
            suite.addTest(case)
        runner = unittest.TextTestRunner(verbosity=2, buffer=True)
        runner.run(suite)
    except Exception as ex:
        print dir(ex)
        print ex.message
        traceback.print_exception(type(ex), ex, ex.__traceback__)
    finally:
        sys.stdout = std_out_origin
        sys.stderr = std_err_origin
    
def get_date_list2(begin_date, end_date, freq):
    date_list = [x for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

test_case_list = [ ] # pass 

begin_date = datetime.datetime(2020, 01, 01)
end_date = datetime.datetime(2020, 05, 31)
date_list = get_date_list2(begin_date, end_date, "W-WED")
#date_list = [datetime.datetime(2020, 4, 8), datetime.datetime(2020, 2, 5), datetime.datetime(2020, 2, 12)]
print date_list
for t_date in date_list:
    test_case_list.append(TestMarketSizeWeekly(trigger_datetime=t_date, methodName='test_market_size_etl_accuracy_and_completeness'))


debug(test_case_list)



In [0]:


import datetime
import unittest

from aadatapipelinecore.core.monitoring.pipeline_monitor import running, fail, task_success
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.utils.identifier import package_id
from aadatapipelinecore.core.utils.spark import canned_spark, stop
from aadatapipelinecore.core.utils import email
from applications.db_check_v1.common.html_report_test_runner import HTMLTestRunner


def get_date_list(begin_date, end_date, freq):
    date_list = [x for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

def send_db_check_email(title, text_context, key=None):
    default_recipients = ['tom@appannie.com']
    email.send(title, text_context, default_recipients, sender='dev-qa-data-quality@appannie.com')


def send_message():
    log_file = "/tmp/db_check.log"
    with open(log_file, "w") as html_file:
        suite = unittest.TestSuite()
        
        # begin_date = datetime.datetime(2020, 01, 01)
        # end_date = datetime.datetime(2020, 05, 20)
        #date_list = get_date_list(begin_date, end_date, "W-WED")

        # date_list = [datetime.datetime(2020, 4, 1), datetime.datetime(2020, 2, 5), datetime.datetime(2020, 1, 29)]
        date_list = [ datetime.datetime(2020, 1, 29)]
        print date_list
        for t_date in date_list:
            suite.addTests([TestMarketSizeWeekly(trigger_datetime=t_date, methodName='test_market_size_etl_accuracy_and_completeness')])
            

        runner = HTMLTestRunner(
            stream=html_file,
            title='DB Test Report',
            description='This db_check the report output by Tech Team.'
        )

        failed_count = 0
        result_list = runner.run(suite).result
        for result in result_list:
            if result[0] == 1 or result[0] == 2:
                failed_count += 1

    with open(log_file, 'r') as html_file:
        today = datetime.date.today()
        str_today = today.strftime("%Y-%m-%d")

        title = "Data Regression Check Report - " + str_today + " - "
        if failed_count == 0:
            title += "Passed"
        else:
            title += "Failed"
        send_db_check_email(title, html_file.read())


send_message()
print 123